<a href="https://colab.research.google.com/github/ykitaguchi77/YOLACT_Mobius/blob/main/YOLOv11_cropeyelid_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inference YOLOv11 segmentation**

https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference

In [ ]:
import cv2
from google.colab import drive
import shutil
import os

#driveのマウント
drive.mount('/content/drive')

!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics

Mounted at /content/drive
Cloning into 'ultralytics'...
remote: Enumerating objects: 45290, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 45290 (delta 129), reused 165 (delta 98), pack-reused 45070 (from 1)
Receiving objects: 100% (45290/45290), 37.84 MiB | 26.20 MiB/s, done.
Resolving deltas: 100% (33706/33706), done.
/content/ultralytics


#**simple inference**

In [ ]:
# YOLOv11
from ultralytics import YOLO
import matplotlib.pyplot as plt

#image_path = r"C:\CorneAI\DemoImage\典型例\【正常】demo008.BMP"
image_path = r"/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_test_images/Control_adult/1000.jpg"

model = YOLO("/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv11/best_458epochs.pt")  # load a pretrained model (recommended for training)
results = model(image_path,save=False)

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

In [ ]:
# 元画像と横並びで表示

from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_single_image(image_path, model_path):
    # モデルの読み込み
    model = YOLO(model_path)

    # 画像の読み込み
    original_img = cv2.imread(image_path)
    if original_img is None:
        print(f"Failed to load image: {image_path}")
        return

    # 推論の実行
    results = model(original_img)

    # マスクの描画（ラベルなし）
    predicted_img = results[0].plot(labels=False, boxes=False)

    # 画像のサイズを揃える
    h1, w1 = original_img.shape[:2]
    h2, w2 = predicted_img.shape[:2]

    max_h = max(h1, h2)
    max_w = max(w1, w2)

    # パディング
    padded_original = np.zeros((max_h, max_w, 3), dtype=np.uint8)
    padded_predicted = np.zeros((max_h, max_w, 3), dtype=np.uint8)

    padded_original[:h1, :w1] = original_img
    padded_predicted[:h2, :w2] = predicted_img

    # 画像を横に連結
    combined_img = np.hstack((padded_original, padded_predicted))

    # Matplotlibで表示
    plt.figure(figsize=(20, 10))
    plt.imshow(cv2.cvtColor(combined_img, cv2.COLOR_BGR2RGB))

    # 凡例を追加（画像と同じ色を使用）
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, fc=(0/255, 0/255, 255/255), alpha=0.6, label='eyelid'),     # 青
        plt.Rectangle((0, 0), 1, 1, fc=(64/255, 224/255, 208/255), alpha=0.6, label='iris'),    # ターコイズ
        plt.Rectangle((0, 0), 1, 1, fc=(128/255, 128/255, 128/255), alpha=0.6, label='pupil')   # グレー
    ]

    plt.legend(
        handles=legend_elements,
        loc='upper right',
        bbox_to_anchor=(0.98, 0.98),
        fontsize=12,
        framealpha=0.8
    )

    plt.axis('off')
    plt.tight_layout()
    plt.show()

# パラメータの設定
model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv11/best_458epochs.pt"
image_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_test_images/Control_adult/1000.jpg"

# 実行
display_single_image(image_path, model_path)

#**Eyecrop**

Haarcascadeで眼周囲を切り抜き（15%マージン）して保存

In [ ]:
## Haarcascedeを用いてシンプルに切り抜き

import cv2
import os
import glob

# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

orig_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_test_images"
dst_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_cropped_images"

# すべてのサブディレクトリを含めて.jpg画像を検索
files = glob.glob(os.path.join(orig_dir, "**/*.jpg"), recursive=True)

for file_path in files:
    # 入力ファイルのパス構造を解析
    relative_path = os.path.relpath(file_path, orig_dir)
    parent_dir = os.path.dirname(relative_path)
    id = os.path.splitext(os.path.basename(file_path))[0]

    # 出力ディレクトリのパスを作成
    out_images_dir = os.path.join(dst_dir, parent_dir, "images")
    out_labels_dir = os.path.join(dst_dir, parent_dir, "labels")

    # 出力ディレクトリが存在しない場合は作成
    os.makedirs(out_images_dir, exist_ok=True)
    os.makedirs(out_labels_dir, exist_ok=True)

    img = cv2.imread(file_path)

    if img is None:
        print(f"Failed to read image: {file_path}")
        continue

    img2 = img.copy()

    # 画像グレースケール化
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # 300pix以上のもので目に見えるものを抽出
    eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))

    # 眼検出判定
    if len(eye_list) >= 1:
        print(f'画像 {id}: 目が{str(len(eye_list))}個検出されました')
    else:
        print(f"画像 {id}: eye detection error")

    # 画像の切り抜きと保存（2個以上検出の時に限る）
    if len(eye_list) >= 2:
        for (ex, ey, ew, eh) in eye_list:
            print(f"img_width: {img2.shape[1]}")
            print("[ex,ey] = %d,%d [ew,eh] = %d,%d" % (ex, ey, ew, eh))
            cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

            # Check if coordinates are within the image bounds
            ey_start = max(int(ey - 0.15 * eh), 0)
            ey_end = min(int(ey + 1.15 * eh), img.shape[0])
            ex_start = max(int(ex - 0.15 * ew), 0)
            ex_end = min(int(ex + 1.15 * ew), img.shape[1])

            # Ensure we have a valid crop area
            if ex_start < ex_end and ey_start < ey_end:
                img_cropped = img[ey_start: ey_end, ex_start: ex_end]

                if ex + eh * 0.5 <= img2.shape[1] / 2:
                    side = "R"  # 横幅の半分より左にあるのは右眼
                else:
                    side = "L"  # 横幅の半分よりより右にあるのは左眼

                print(f"side: {side}")
                print("")

                # 出力ファイル名
                output_filename = f"{id}_{side}"

                # 画像とラベルの保存
                cv2.imwrite(os.path.join(out_images_dir, f"{output_filename}.png"), img_cropped)

                # YOLOのラベルを記載したtextファイルを作成
                label_path = os.path.join(out_labels_dir, f"{output_filename}.txt")
                with open(label_path, "w") as f:
                    if side == "R":
                        label = 0
                    else:
                        label = 1

                    # YOLO座標の計算
                    yolo_x = (ex_start + ex_end) / (2 * img.shape[1])
                    yolo_y = (ey_start + ey_end) / (2 * img.shape[0])
                    yolo_w = (ex_end - ex_start) / img.shape[1]
                    yolo_h = (ey_end - ey_start) / img.shape[0]

                    f.write(f"{label} {yolo_x:.6f} {yolo_y:.6f} {yolo_w:.6f} {yolo_h:.6f}")
            else:
                print(f"画像 {id}: Invalid crop coordinates. Skipping.")
    else:
        print(f"画像 {id}: Less than 2 eyes detected. Skipping.")

In [ ]:
## Haarcascadeで切り抜いた画像をYOLOでさらに切り抜き。切り抜きはうまくいっているがlabelの座標がうまくいっていない
"""
To do
セグメンテーションを輪郭検出→座標を取る→YOLOv11のlabelに変換
"""

import cv2
import os
import glob
import numpy as np
from ultralytics import YOLO

# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'
model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv11/best_458epochs.pt"

# カスケード分類器とYOLOモデルの読み込み
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)
yolo_model = YOLO(model_path)

orig_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_test_images"
dst_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_cropped_images"

# すべてのサブディレクトリを含めて.jpg画像を検索
files = glob.glob(os.path.join(orig_dir, "**/*.jpg"), recursive=True)

for file_path in files:
    print(f"\n処理中の画像: {file_path}")

    # 入力ファイルのパス構造を解析
    relative_path = os.path.relpath(file_path, orig_dir)
    parent_dir = os.path.dirname(relative_path)
    id = os.path.splitext(os.path.basename(file_path))[0]

    # サブディレクトリを含む出力ディレクトリのパスを作成
    sub_dir = os.path.join(dst_dir, parent_dir)
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)

    # 画像、ラベル、マスク用のサブディレクトリを作成
    out_images_dir = os.path.join(sub_dir, "images")
    out_labels_dir = os.path.join(sub_dir, "labels")
    out_masks_dir = os.path.join(sub_dir, "masks")

    # 各サブディレクトリを作成（exist_ok=Trueを追加）
    for dir_path in [out_images_dir, out_labels_dir, out_masks_dir]:
        os.makedirs(dir_path, exist_ok=True)

    img = cv2.imread(file_path)

    if img is None:
        print(f"Failed to read image: {file_path}")
        continue

    img2 = img.copy()

    # 画像グレースケール化
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # 300pix以上のもので目に見えるものを抽出
    eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))

    # 眼検出判定
    if len(eye_list) >= 1:
        print(f'画像 {id}: 目が{str(len(eye_list))}個検出されました')
    else:
        print(f"画像 {id}: eye detection error")

    # 画像の切り抜きと保存（2個の場合のみ処理）
    if len(eye_list) == 2:
        for (haar_x, haar_y, haar_w, haar_h) in eye_list:
            print(f"img_width: {img2.shape[1]}")
            print("[haar_x,haar_y] = %d,%d [haar_w,haar_h] = %d,%d" % (haar_x, haar_y, haar_w, haar_h))

            # Check if coordinates are within the image bounds
            ey_start = max(int(haar_y - 0.15 * haar_h), 0)
            ey_end = min(int(haar_y + 1.15 * haar_h), img.shape[0])
            ex_start = max(int(haar_x - 0.15 * haar_w), 0)
            ex_end = min(int(haar_x + 1.15 * haar_w), img.shape[1])

            # Ensure we have a valid crop area
            if ex_start < ex_end and ey_start < ey_end:
                img_cropped = img[ey_start: ey_end, ex_start: ex_end]

                if haar_x + haar_h * 0.5 <= img2.shape[1] / 2:
                    side = "R"  # 横幅の半分より左にあるのは右眼
                else:
                    side = "L"  # 横幅の半分よりより右にあるのは左眼

                print(f"side: {side}")
                print("")

                # 出力ファイル名
                output_filename = f"{id}_{side}"

                # YOLOv11でinference実行
                results = yolo_model(img_cropped)

                # YOLOv11の検出結果からeyelidの横幅を取得
                eyelid_box = None
                for result in results:
                    boxes = result.boxes
                    for box in boxes:
                        cls_id = int(box.cls)
                        if cls_id == 0:  # eyelidのクラスID = 0
                            yolo_x1, yolo_y1, yolo_x2, yolo_y2 = box.xyxy[0]  # xyxy形式で座標を取得
                            yolo_width = yolo_x2 - yolo_x1  # 実際のピクセル幅
                            if eyelid_box is None or yolo_width > eyelid_box[2]:
                                eyelid_box = (yolo_x1, yolo_y1, yolo_width, yolo_y2 - yolo_y1)

                if eyelid_box is not None:
                    yolo_x1, yolo_y1, yolo_width, yolo_height = eyelid_box
                    # eyelidの横幅を基準に正方形サイズを計算
                    square_size = int(yolo_width * 1.5)  # padding 25% × 2 = 50% 増

                    # 正方形の中心をeyelidの中心に合わせる
                    center_x = yolo_x1 + yolo_width/2
                    center_y = yolo_y1 + yolo_height/2

                    # 正方形の領域を計算
                    square_x1 = int(max(center_x - square_size/2, 0))
                    square_y1 = int(max(center_y - square_size/2, 0))
                    square_x2 = int(min(square_x1 + square_size, img_cropped.shape[1]))
                    square_y2 = int(min(square_y1 + square_size, img_cropped.shape[0]))

                    # 正方形領域を切り出し
                    img_square = img_cropped[square_y1:square_y2, square_x1:square_x2]

                    # もし切り出した領域が正方形でない場合（端の処理）
                    actual_size = max(img_square.shape[0], img_square.shape[1])
                    square_img = np.zeros((actual_size, actual_size, 3), dtype=np.uint8)
                    y_offset = (actual_size - img_square.shape[0]) // 2
                    x_offset = (actual_size - img_square.shape[1]) // 2
                    square_img[y_offset:y_offset+img_square.shape[0],
                             x_offset:x_offset+img_square.shape[1]] = img_square

                    # 切り抜き画像の保存
                    output_image_path = os.path.join(out_images_dir, f"{output_filename}.png")
                    cv2.imwrite(output_image_path, square_img)

                    # マスク画像を生成（ラベルとボックスなし）
                    mask_img = results[0].plot(labels=False, boxes=False)

                    # マスク画像も同じ正方形領域で切り出し
                    mask_square = mask_img[square_y1:square_y2, square_x1:square_x2]

                    # マスク画像も同様にパディング処理
                    square_mask = np.zeros((actual_size, actual_size, 3), dtype=np.uint8)
                    square_mask[y_offset:y_offset+mask_square.shape[0],
                              x_offset:x_offset+mask_square.shape[1]] = mask_square

                    # マスク画像の保存
                    cv2.imwrite(os.path.join(out_masks_dir, f"{output_filename}_mask.png"), square_mask)

                # YOLOのラベルを記載したtextファイルを作成
                label_path = os.path.join(out_labels_dir, f"{output_filename}.txt")
                with open(label_path, "w") as f:
                    # Haarcascadeで検出された目の座標をすべて書き込み
                    for i, (haar_eye_x, haar_eye_y, haar_eye_w, haar_eye_h) in enumerate(eye_list):
                        eye_label = i  # 0: 右目, 1: 左目
                        # もとの画像に対する相対座標を計算
                        yolo_x = (haar_eye_x + haar_eye_w/2) / img.shape[1]  # 中心のx座標
                        yolo_y = (haar_eye_y + haar_eye_h/2) / img.shape[0]  # 中心のy座標
                        yolo_w = haar_eye_w / img.shape[1]  # 幅
                        yolo_h = haar_eye_h / img.shape[0]  # 高さ

                        # Haarcascadeの検出結果を書き込み
                        f.write(f"{eye_label} {yolo_x:.6f} {yolo_y:.6f} {yolo_w:.6f} {yolo_h:.6f}\n")

                    # YOLOv11の検出結果からeyelidの横幅を取得
                    eyelid_box = None
                    for result in results:
                        boxes = result.boxes
                        for box in boxes:
                            cls_id = int(box.cls)
                            if cls_id == 0:  # eyelidのクラスID = 0
                                yolo_x1, yolo_y1, yolo_x2, yolo_y2 = box.xyxy[0]  # xyxy形式で座標を取得
                                yolo_width = yolo_x2 - yolo_x1  # 実際のピクセル幅
                                if eyelid_box is None or yolo_width > eyelid_box[2]:
                                    eyelid_box = (yolo_x1, yolo_y1, yolo_width, yolo_y2 - yolo_y1)

                    if eyelid_box is not None:
                        yolo_x1, yolo_y1, yolo_width, yolo_height = eyelid_box
                        # eyelidの横幅を基準に正方形サイズを計算
                        square_size = yolo_width * 1.5  # padding 25% × 2 = 50% 増

                        # YOLOv11の検出結果を正方形サイズで正規化
                        for result in results:
                            boxes = result.boxes
                            for box in boxes:
                                cls_id = int(box.cls)
                                yolo_box_x1, yolo_box_y1, yolo_box_x2, yolo_box_y2 = box.xyxy[0]  # xyxy形式で座標を取得

                                # 中心座標とサイズを計算（正方形の中心を基準に）
                                center_x = (yolo_box_x1 + yolo_box_x2) / 2
                                center_y = (yolo_box_y1 + yolo_box_y2) / 2
                                box_w = yolo_box_x2 - yolo_box_x1
                                box_h = yolo_box_y2 - yolo_box_y1

                                # eyelidの中心を正方形の中心（0.5, 0.5）とした相対座標に変換
                                relative_x = 0.5 + ((center_x - (yolo_x1 + yolo_width/2)) / square_size)
                                relative_y = 0.5 + ((center_y - (yolo_y1 + yolo_height/2)) / square_size)
                                relative_w = box_w / square_size
                                relative_h = box_h / square_size

                                # YOLOv11の検出結果を追記
                                f.write(f"{cls_id} {relative_x:.6f} {relative_y:.6f} {relative_w:.6f} {relative_h:.6f}\n")
                    else:
                        print(f"Warning: No eyelid detected in {output_filename}")
            else:
                print(f"画像 {id}: Invalid crop coordinates. Skipping.")
    else:
        print(f"画像 {id}: {len(eye_list)}個の目が検出されたため、スキップします。")

#**mask --> points**